In [46]:
from environments import MountainCar
from features import LinearFeatureMap
import numpy as np
from tqdm.notebook import tqdm

In [41]:
#tuples = np.load('mc_data.npy',allow_pickle = True)

In [44]:
H = 150
num_trials = 1000000
env = MountainCar(H)

s = np.zeros((2,num_trials))
s[0,:] = np.random.uniform(low = -1.2,high = 0.1,size = num_trials)
env.reset()
tuples = []

for h in tqdm(range(H)):
    a = np.random.choice([-1,0,1],size=num_trials)
    cost, s_ = env.step_broadcast(s, a, num_trials)
    tuples.append([s,a+1,cost,s_,h])
    s = s_


  0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
class FittedQIteration(object):
    def __init__(self,phi,features,data,horizon,k):
        self.phi = phi
        self.features = features
        self.data = data
        self.H = horizon
        self.n = len(data)
        self.k = k
        self.d = len(self.get_phi(np.array([0,0])))
        self.theta_ = np.zeros((self.H,3,self.d)) 
        self.theta = np.zeros((self.H,3,self.d))
        self.A = np.zeros((self.H,3,self.d,self.d))
        self.A[:] = np.identity(self.d) 
        self.get_A()
        
    
    def get_phi(self,state):
        if self.features == 'poly':
            return self.phi.polynomial_basis(state)
        elif self.features == 'fourier':
            phi = self.phi.fourier_basis(state)
            return phi[:,0]
        
    
    def get_A(self):
        print('Getting A')
        data = self.data.copy()
        for i in tqdm(range(self.n)):
            s,a,h = data[i][0], data[i][1], data[i][4]
            phi = self.get_phi(s)
            self.A[h,a] += np.outer(phi,phi)
    
    
    def get_targets(self,h):
        data = self.data.copy()
        self.tar = np.zeros((self.d))
        for i in range(self.n):
            s, a, c, s_, h = data[i][0], data[i][1], data[i][2], data[i][3], data[i][4]
            if s_ != None:
                phi = self.get_phi(s)
                phi_ = self.get_phi(s_)
                q = np.zeros(3)
                for a_ in range(3):
                    q[a_] = np.inner(phi_, self.theta_[h+1,a_])
                v = np.min(q)
                self.tar[h,a] += max(0,min((c + v),1)) * phi
            else:
                self.tar[h,a] += c * phi
        
     
    def minimize_sq(self):
        for h in range(self.H-1,-1,-1):
            for a in range(3):
                self.theta[h,a] = np.linalg.solve(self.A[h,a],self.tar[h,a])
            
    
    def update_Q(self):
        self.get_targets()
        self.minimize_sq()
        self.theta_ = self.theta
        
    
    
    def run(self):
        print('Running')
        for t in tqdm(range(self.k)):
            self.update_Q()
            q = np.zeros(3)
            s = np.array([0.5,0])
            phi = self.get_phi(s)
            for a in range(3):
                q[a] = np.inner(phi,self.theta[0,a])
            print(q)
        

In [ ]:
for i in [1000,5000,10000,20000,40000,80000,100000,1000000]:
    phi = LinearFeatureMap()
    #phi.init_poly_features(2,1)
    phi.init_fourier_features(2,4)
    phi.init_state_normalizers(np.array([0.6,0.07]),np.array([-1.2,-0.07]))
    features = 'fourier'
    print(i)
    agent = LeastSquaresFittedQIteration(phi,features,tuples[:i],H,10)
    agent.run()

In [37]:
costs = tuples[-1][2]

In [38]:
x = np.where(costs==0.0)

In [39]:
len(x[0])/num_trials

0.000443

In [24]:
x

(array([    11,     26,     72, ..., 999970, 999993, 999996]),)

In [51]:
phi = LinearFeatureMap()
phi.init_fourier_features(2,4)
phi.init_state_normalizers(np.array([0.6,0.07]),np.array([-1.2,-0.07]))

In [52]:
s = np.zeros((2,5))
s[0,:] = 0.5

In [55]:

phi.fourier_basis(s)

ValueError: operands could not be broadcast together with shapes (2,5) (2,) 